In [67]:
import numpy as np
import pandas as pd
from scipy import integrate

In [68]:
def phi(x):
    return np.sin(3 * np.pi * x)

def psi(x, p):
    return np.sqrt(2) * np.sin(p* np.pi * x)


def create_table(w_uf, N, M):
    x_arr = np.linspace(0, 1, N + 1)
    t_arr = np.linspace(0, 0.1, M + 1)
    columns = []
    for x in x_arr:
        columns.append("x={r}".format(r=x))
    grid_table = pd.DataFrame(data=w_uf, index=t_arr, columns=columns)
    grid_table.columns.name = "t "
    return grid_table


def error(u, uf, N, M):
    err = []
    for k in range (M + 1):
        for i in range (N + 1):
            err.append(abs(u[k][i] - uf[k][i]))
    return max(err)

N = 5
M = 5
h = 1.0 / N
tau = 0.1 / M

x_arr = np.linspace(0, 1, 6)
t_arr = np.linspace(0, 0.1, 6)
P = 20

In [69]:
def Fourier_solve(cp, P, x, t):
    res = 0
    for p in range(P - 1):
        res += cp[p] * np.exp(-np.pi ** 2 * (p + 1) ** 2 * t) * psi(x, p + 1)
    return res

Значения точного uf решения с использованием ряда Фурье.

In [70]:
def uf(N, M):
    x_arr = np.linspace(0, 1, N + 1)
    t_arr = np.linspace(0, 0.1, M + 1)
    cp = np.zeros(P - 1)
    for p in range(1, P):
        cp[p - 1] = integrate.quad(lambda x: phi(x) * psi(x, p), 0, 1)[0]
    np.around(cp, 3)

    uf = np.zeros((M + 1, N + 1))
    for i in range(M + 1):
        for j in range(N + 1):
            uf[i][j] = Fourier_solve(cp, P, x_arr[j], t_arr[i])
    return uf

uf_ = uf(5, 5)
uf_table = create_table(uf_, 5, 5)
uf_table

t,x=0.0,x=0.2,x=0.4,x=0.6000000000000001,x=0.8,x=1.0
0.00,0.0,0.951057,-0.587785,-0.587785,0.951057,3.673940e-16
0.02,0.0,0.160942,-0.099468,-0.099468,0.160942,6.217209e-17
0.04,0.0,0.027235,-0.016832,-0.016832,0.027235,1.052104e-17
0.06,0.0,0.004609,-0.002848,-0.002848,0.004609,1.780419e-18
0.08,0.0,0.000780,-0.000482,-0.000482,0.000780,3.012905e-19
0.10,0.0,0.000132,-0.000082,-0.000082,0.000132,5.098575e-20


Значения решения с использованием дискретного ряда Фурье.

In [71]:
def d_uf(N_):
    h = 1 / N_
    cp_duf = np.zeros(N_ - 1)
    for p in range(1, N_):
        cp_duf[p - 1] = h * sum([phi(i * h) * psi(i * h, p) for i in range(1, N_)])

    duf = np.zeros((M + 1, N + 1))
    for i in range(M + 1):
        for j in range(N + 1):
            duf[i][j] = Fourier_solve(cp_duf, N, x_arr[j], t_arr[i])
    return duf

duf = d_uf(5)
duf_table = create_table(duf, 5, 5)
duf_table

t,x=0.0,x=0.2,x=0.4,x=0.6000000000000001,x=0.8,x=1.0
0.00,0.0,0.951057,-0.587785,-0.587785,0.951057,3.673940e-16
0.02,0.0,0.160942,-0.099468,-0.099468,0.160942,6.217209e-17
0.04,0.0,0.027235,-0.016832,-0.016832,0.027235,1.052104e-17
0.06,0.0,0.004609,-0.002848,-0.002848,0.004609,1.780419e-18
0.08,0.0,0.000780,-0.000482,-0.000482,0.000780,3.012905e-19
0.10,0.0,0.000132,-0.000082,-0.000082,0.000132,5.098575e-20


In [72]:
error_5 = error(duf, uf_, 5, 5)
error_5

1.887379141862766e-15

In [73]:
error_10 = error(d_uf(10), uf(5, 5), 5, 5)
error_10

1.6653345369377348e-15

In [74]:
error_20 = error(d_uf(20), uf(5, 5), 5, 5)
error_20

1.3322676295501878e-15

Значения сеточного решения с использованием ДРФ при различных параметрах sigma

In [75]:
def lambdas(param, N, M):
    h = 1.0 / N
    tau = 0.1 / M
    l = []
    for p in range(1, N):
        tmp1 = 1.0 - (4.0 * (1.0 - param) * tau / h ** 2.0) * np.sin(p * np.pi * h / 2.0) ** 2
        tmp2 = 1.0 + (4.0 * param * tau / h ** 2.0) * np.sin(p * np.pi * h / 2.0) ** 2
        l.append(tmp1 / tmp2)
    return l

def grid_Fourier_solve(cp, l, x, t, N, M):
    tau = 0.1 / M
    k = t / tau
    return sum(cp[p] * (l[p] ** k) * psi(x, p + 1) for p in range(N - 1))


def grid_param(sigma, N, M):
    h = 1.0 / N
    x_arr = np.linspace(0, 1, N + 1)
    t_arr = np.linspace(0, 0.1, M + 1)
    cp_wght = np.zeros(N - 1)
    for p in range(1, N):
        cp_wght[p - 1] = h * sum([phi(i * h) * psi(i * h, p) for i in range (1, N)])
    l = lambdas(sigma, N, M)

    w_uf = np.zeros((M + 1, N + 1))
    for i in range(M + 1):
        for j in range(N + 1):
            w_uf[i][j] = grid_Fourier_solve(cp_wght, l, x_arr[j], t_arr[i], N, M)
    return w_uf

In [76]:
N_M = ((5, 5), (10, 20), (20, 80), (20, 20))
sigmas = (0, 0.5, 1)
result = []
for i in range(len(N_M)):
    arr_result = []
    for j in range(len(sigmas)):
        arr_result.append(error(grid_param(sigmas[j], N_M[i][0], N_M[i][1]), uf(N_M[i][0], N_M[i][1]), N_M[i][0], N_M[i][1]))
    result.append(arr_result)
g_uf_0 = grid_param(0.5, 5, 5)

result = np.array(result).transpose()
error_grid_table = pd.DataFrame(data = result, index = ['σ = 0', 'σ = 1/2', 'σ = 1'], columns = ['(0.2, 0.02)', '(0.1, 0.005)', '(0.05, 0.00125)','(0.05, 0.005)'])
error_grid_table.columns.name = "(h, t)"
error_grid_table

/var/folders/25/hkg7hrfn7130z9vkbc1z1h580000gn/T/ipykernel_2365/2801451979.py:14: RuntimeWarning: invalid value encountered in double_scalars
  return sum(cp[p] * (l[p] ** k) * psi(x, p + 1) for p in range(N - 1))


"(h, t)","(0.2, 0.02)","(0.1, 0.005)","(0.05, 0.00125)","(0.05, 0.005)"
σ = 0,0.454835,0.065878,0.014199,16.004979
σ = 1/2,0.037656,0.021894,0.006455,0.000864
σ = 1,0.250946,0.091213,0.025992,0.073879
